# Short Time Series Biclustering

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import glob
import multiprocessing
from operator import attrgetter
from sklearn.datasets import make_biclusters
from sklearn.datasets import samples_generator as sg
from sklearn.preprocessing import MinMaxScaler

In [32]:
class BisectingClusterer(object):
    def __init__(self, data):
        if data is not None:
            self._data = np.array(data)
            self._I, self._J = self._data.shape
        else:
            print("Empty data")
    
    @property
    def centroids(self):
        return self._centroids

    def fit(self):
        self._centroids = self._compute_centroids_()
        bisecting_indices = self._bisect_clusters_(self._centroids)
        return bisecting_indices
    
    def fit_rows(self):
        return self.fit()
    
    def fit_cols(self):
        data = self._data.T
        aux_I, aux_J = data.shape
        assert aux_I >= 3, "bicluster not large enough"
        aux_indices = np.zeros(aux_I)
        print(len(data))
        for i in range (1, aux_I-1):
            print(i)
            print(data[i:len(data)-2])
        
    def _compute_centroids_(self):
        max_correlation = 0
        centroids = [0,0]
        for i in range(self._I):
            for j in range(i+1, self._I):
                if (i == j):
                    break
                correlation = PositiveNegativeCorrelation(self._data[i],
                                                          self._data[j],
                                                          self._J).H_neg
                if(correlation > max_correlation):
                    max_correlation = correlation
                    centroids[0] = i
                    centroids[1] = j
        return centroids

    def _bisect_clusters_(self, centroids):
        cluster_indices = np.zeros(self._I)
        for i in range(self._I):
            correlation0 = PositiveNegativeCorrelation(
                self._data[centroids[0]], self._data[i],self._J).H_pos
            correlation1 = PositiveNegativeCorrelation(
                self._data[centroids[1]], self._data[i],self._J).H_pos
            if(correlation0 <= correlation1):
                cluster_indices[i] = 1
        return cluster_indices

In [2]:
%%latex
Define H for a new term 'z' in X
$$
H_{1}(I,J,X,z) =H_{0}(I,J,X) \cdot \frac{(I-1)}{(I+1)} + \frac{|2|}{(I)(I+1)} \cdot \sum_{x \epsilon X} {h(x,z,J)}
$$

<IPython.core.display.Latex object>

In [3]:
class ClusterMaps(object):
    def __init__(self, rho=None, gamma=None, H = 0):
        self.gamma = gamma
        self.rho = rho
        self.H = H
    
    def __str__(self):
        return ("rho: \n"+ str(self.rho) + "\ngamma: \n" + str(self.gamma) + "\nH: \n " + str(self.H))

In [38]:
test_arr = np.random.rand(5,5)
test_arr

array([[ 0.50095512,  0.21679472,  0.77068066,  0.6415653 ,  0.88970651],
       [ 0.32320885,  0.57973748,  0.83682875,  0.14483868,  0.16475132],
       [ 0.94263798,  0.13595588,  0.99100005,  0.2959615 ,  0.87292524],
       [ 0.52516041,  0.31402082,  0.07313222,  0.08408673,  0.64547762],
       [ 0.04454499,  0.75482119,  0.0262751 ,  0.16758357,  0.11956992]])

In [39]:
test_arr.T[3:4]

array([[ 0.6415653 ,  0.14483868,  0.2959615 ,  0.08408673,  0.16758357]])

In [34]:
cl_test = BisectingClusterer(test_arr)

In [35]:
cl_test.fit_cols()

5
1
[[ 0.32596224  0.44816576  0.24778532  0.76785447  0.45158472]
 [ 0.5376395   0.59762419  0.72751057  0.9863876   0.24581147]]
2
[[ 0.5376395   0.59762419  0.72751057  0.9863876   0.24581147]]
3
[]


In [ ]:
class TsRapooc(object):
    def __init__(self, data, n_clusters = 5):
        self._D = np.array(data)
        self._n_clusters = n_clusters
        self._labels = list()
        self._I, self._J = data.shape
        
    @property
    def n_clusters(self):
        return self._n_clusters
    
    @property
    def data(self):
        return self._D
    
    @property
    def labels(self):
        return _labels